First we import the libraries we will be using throughout the implementation

In [1]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, Flatten, Lambda, Concatenate, concatenate
from keras.optimizers import Adam
from matplotlib import pyplot as plt

We define some functions in order to load our dataset, we will use them later in our code 

In [2]:
#Function used to load the class ids for the images
def load_class_ids(class_info_file_path):
    with open(class_info_file_path,'rb') as f:
        class_ids= pickle.load(f,encoding='latin1')
        return class_ids


#Function used to load the embeddings for the images
def load_embeddings(embeddings_file_path):
    with open(embeddings_file_path,'rb') as f:
        embeddings= pickle.load(f,encoding='latin1')
        embeddings= np.array(embeddings)
        print('Embeddings: ',embeddings.shape)
    return embeddings


#Function used to load the filenames for the images
def load_filenames(filename_file_path):
    with open(filename_file_path,'rb')as f:
        filenames= pickle.load(f,encoding='latin1')
    return filenames


#Function used to load the bounding boxes for the images, and also to make a dictionary out of it
def load_bounding_boxes(dataset_dir):
    # Paths of both the files
    bounding_boxes_path= os.path.join(dataset_dir, 'bounding_boxes.txt')
    file_paths_path= os.path.join(dataset_dir, 'images.txt')

    #Read the files, making dataframes of the data
    df_bounding_boxes= pd.read_csv(bounding_boxes_path, delim_whitespace=True, header=None).astype(int)
    df_file_names= pd.read_csv(file_paths_path, delim_whitespace=True, header=None)

    #Create a list of filenames
    file_names= df_file_names[1].tolist()

    #Create a dictionary, filename corresponds to its respective bounding box
    filename_bbox_dict= {img_file[:-4]:[] for img_file in file_names[:2]}

    #Assign a bounding box to the corresponding image
    for i in range(0,len(file_names)):
        bounding_box= df_bounding_boxes.iloc[i][1:].tolist()
        key= file_names[i][:-4]
        filename_bbox_dict[key]= bounding_box

    return filename_bbox_dict


#Function used to load the images and also cut them into the required size around their bounding box
def get_img(img_path, bbox, image_size):
    img= Image.open(img_path).convert('RGB')
    width,height= img.size
    if bbox is not None:
        R= int(np.maximum(bbox[2],bbox[3])*0.75) # R refers to the radius of the square region of interest (ROI)
        center_x= int((2*bbox[0]+bbox[2])/2)
        center_y= int((2*bbox[1]+bbox[3])/2)
        y1= np.maximum(0,center_y-R)
        y2= np.minimum(height,center_y+R)
        x1= np.maximum(0,center_x-R)
        x2= np.minimum(width,center_x+R)
        img=img.crop([x1,y1,x2,y2])
    img= img.resize(image_size, PIL.Image.BILINEAR)
    return img


#Function used to load the entire dataset, and returns x:images as array, y:class ids of the images, embedding: embeddings of the images
def load_dataset(filenames_file_path, class_info_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    filenames =load_filenames(filenames_file_path)
    class_ids= load_class_ids(class_info_file_path)
    bounding_boxes= load_bounding_boxes(cub_dataset_dir)
    all_embeddings= load_embeddings(embeddings_file_path)

    x,y,embeddings= [],[],[]

    print("Embeddings Shape: ",all_embeddings.shape)

    for index,filename in enumerate(filenames):
        bounding_box= bounding_boxes[filename]

        try:
            img_name= '{}/images/{}.jpg'.format(cub_dataset_dir,filename)
            img= get_img(img_name,bounding_box,image_size)

            all_embeddings1= all_embeddings[index,:,:]

            embedding_ix= random.randint(0,all_embeddings1.shape[0]-1)
            embedding= all_embeddings1[embedding_ix,:]

            x.append(np.array(img))
            y.append(class_ids[index])
            embeddings.append(embedding)
        except Exception as e:
            print(e)
    
    x=np.array(x)
    y=np.array(y)
    embeddings= np.array(embeddings)
    return x,y,embeddings

Now we will build the Stage 1 architecture for the StackGAN

In [3]:
# A function used to generate the c value using the reparameterization trick
def generate_c(x):
    mean=x[:,:128]
    log_sigma= x[:,128:]
    stddev= K.exp(log_sigma)
    epsilon= K.random_normal(shape=K.constant((mean.shape[1],),dtype='int32'))
    c= stddev * epsilon + mean
    return c


# Build the Conditional Augmentation model
# Takes as input an embedding of shape (1024,) and returns a tensor of shape (256,)
# Its primary purpose is to prevent mode collapses and encourage the generator to produce diverse outputs for a given condition
# The generator is able to produce diverse images for the same given prompt 
def build_ca_model():
    input_layer= Input(shape=(1024,))
    x= Dense(256)(input_layer)
    x= LeakyReLU(alpha=0.2)(x)
    model= Model(inputs= [input_layer], outputs=[x])
    return model



# Building the Embedding Compressor Model
def build_embedding_compressor_model():
    input_layer= Input(shape=(1024,))
    x= Dense(128)(input_layer)
    x= ReLU()(x)
    model= Model(inputs=[input_layer],outputs=[x])
    return model

We are onto a Major step in the Stage 1 for the StackGAN

We are now building the stage 1 generator for the model

In [4]:
# Builds the generator model used in Stage 1
def build_stage1_generator():
    input_layer= Input(shape=(1024,))
    x= Dense(256)(input_layer)
    mean_logsigma= LeakyReLU(alpha=0.2)(x)

    c= Lambda(generate_c)(mean_logsigma)

    input_layer2= Input(shape=(100,))
    gen_input= Concatenate(axis=1)([c,input_layer2])

    x=Dense(128*8*4*4, use_bias=False)(gen_input)
    x=ReLU()(x)

    x=Reshape((4,4,128*8), input_shape= (128*8*4*4,))(x)

    x=UpSampling2D(size=(2,2))(x)
    x= Conv2D(filters=512, kernel_size=3, padding='same',strides=1,use_bias=False)(x)
    x=BatchNormalization()(x)
    x= ReLU()(x)

    x=UpSampling2D(size=(2,2))(x)
    x= Conv2D(filters=256, kernel_size=3, padding='same',strides=1,use_bias=False)(x)
    x=BatchNormalization()(x)
    x= ReLU()(x)

    x=UpSampling2D(size=(2,2))(x)
    x= Conv2D(filters=128, kernel_size=3, padding='same',strides=1,use_bias=False)(x)
    x=BatchNormalization()(x)
    x= ReLU()(x)

    x=UpSampling2D(size=(2,2))(x)
    x= Conv2D(filters=64, kernel_size=3, padding='same',strides=1,use_bias=False)(x)
    x=BatchNormalization()(x)
    x= ReLU()(x)

    x= Conv2D(filters=3,kernel_size=3,padding='same', strides=1, use_bias=False)(x)
    x= Activation(activation='tanh')(x)

    stage1_gen= Model(inputs= [input_layer,input_layer2], outputs=[x,mean_logsigma])
    return stage1_gen

We will now build the Stage 1 Discriminator

In [5]:
"""
Creating a Model which takes 2 inputs
1. One from the Generator
2. One from the embedding layer
3. Concatenate along the axis dimension and feed it to the last module which produces final logits
"""
def build_stage1_discriminator():
    input_layer= Input(shape=(64,64,3))

    x= Conv2D(filters=64, kernel_size=4, padding='same', strides=2 ,input_shape= (64,64,3), use_bias=False)(input_layer)
    x=LeakyReLU(alpha=0.2)(x)

    x= Conv2D(filters=128,kernel_size= 4,strides= 2,padding= 'same',use_bias=False)(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)

    x= Conv2D(filters=256,kernel_size= 4,strides= 2,padding= 'same',use_bias=False)(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)

    x= Conv2D(filters=512,kernel_size= 4,strides= 2,padding= 'same',use_bias=False)(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)

    input_layer2= Input(shape=(4,4,128))

    merged_input= concatenate([x,input_layer2])

    x2= Conv2D(64*8, 1,1, 'same')(merged_input)
    x2= BatchNormalization()(x2)
    x2=LeakyReLU(0.2)(x2)
    x2=Flatten()(x2)
    x2= Dense(1)(x2)
    x2= Activation('sigmoid')(x2)

    stage1_dis= Model(inputs= [input_layer, input_layer2], outputs=[x2])
    return stage1_dis

In [6]:
def build_adversarial_model(gen_model, dis_model):
    input_layer= Input(shape=(1024,))
    input_layer2= Input(shape= (100,))
    input_layer3= Input(shape=(4,4,128))

    x,mean_logsigma= gen_model([input_layer, input_layer2])

    dis_model.trainable= False
    valid= dis_model([x,input_layer3])

    model= Model(inputs=[input_layer,input_layer2,input_layer3], outputs=[valid, mean_logsigma])
    return model

Now we write a function for our KL Loss

In [7]:
# Function to calculate the Kullback_Leibler divergence loss (KL Loss)
def KL_loss(ytrue,ypred):
    mean=ypred[:,:128]
    logsigma= ypred[:,128:]
    loss= -logsigma + 0.5*(-1 + K.exp(2. * logsigma) + K.square(mean))
    loss=K.mean(loss)
    return loss

def custom_generator_loss(ytrue,ypred):
    return K.binary_crossentropy(ytrue,ypred)

Now, we write some function for saving the generated image after every 2 epochs and saving the logs.

In [8]:
# Function for saving the rgb image to the specified path
def save_rgb_img(img,path):
    fig= plt.figure()
    ax= fig.add_subplot(1,1,1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()


# Function to write training summary to the TensorBoard
def write_log(callback, name,loss,batch_no):
    with tf.summary.create_file_writer('logs').as_default():
        tf.summary.scalar(name,loss,batch_no)

Let’s write the main function to initialize the hyper-parameters and train our Stage 1 StackGAN.

In [9]:
if __name__== '__main__':
    # Defining the variables
    data_dir= "/birds/"
    train_dir= data_dir + "/train"
    test_dir= data_dir + "/test"
    image_size= 64
    batch_size= 64
    z_dim= 100
    stage1_generator_lr= 0.0002
    stage1_discriminator_lr= 0.0002
    stage1_lr_decay_step = 600
    epochs=1000
    condition_dim= 128

    # Loading datasets 
    embeddings_file_path_train= "birds/train/char-CNN-RNN-embeddings.pickle"
    embeddings_file_path_test= "birds/test/char-CNN-RNN-embeddings.pickle"

    filenames_file_path_train= "birds/train/filenames.pickle"
    filenames_file_path_test= "birds/test/filenames.pickle"

    class_info_file_path_train = "birds/train/class_info.pickle"
    class_info_file_path_test = "birds/test/class_info.pickle"

    cub_dataset_dir= "CUB_200_2011"

    # Defining the Optimizers for the models 
    dis_optimizer= Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer= Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

    #Load the datasets 
    xtrain, ytrain, embeddings_train= load_dataset(filenames_file_path_train, class_info_file_path_train, cub_dataset_dir, embeddings_file_path_train, (64,64))
    xtest,ytest,embeddings_test= load_dataset(filenames_file_path_test,class_info_file_path_test, cub_dataset_dir, embeddings_file_path_test, (64,64))

    # Building and Compiling the network
    ca_model= build_ca_model()
    ca_model.compile(loss= 'binary_crossentropy', optimizer='adam')

    stage1_dis= build_stage1_discriminator()
    stage1_dis.compile(loss='binary_crossentropy',optimizer=dis_optimizer)

    stage1_gen= build_stage1_generator()
    stage1_gen.compile(loss='mse',optimizer=gen_optimizer)

    embedding_compressor_model= build_embedding_compressor_model()
    embedding_compressor_model.compile(loss= 'binary_crossentropy',optimizer='adam')

    adversarial_model= build_adversarial_model(stage1_gen, stage1_dis)
    adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1,2.0], optimizer=gen_optimizer, metrics=None)

    tensorboard= TensorBoard(log_dir="logs/".format(time.time()))
    tensorboard.set_model(stage1_gen)
    tensorboard.set_model(stage1_dis)
    tensorboard.set_model(ca_model)
    tensorboard.set_model(embedding_compressor_model)

    # Generate an array containing real and fake values 
    # Apply label smoothing as well
    real_labels= np.ones((batch_size,1), dtype=float)*0.9
    fake_labels= np.zeros((batch_size,1), dtype=float)*0.1

    for epoch in range(epochs):
        print("===============================")
        print("Epoch: ", epoch)
        print("Number of Batches: ", int(xtrain.shape[0]/batch_size))

        gen_losses=[]
        dis_losses=[]

        #Loading the data and training the model
        number_of_batches= int(xtrain.shape[0]/batch_size)
        for index in range(number_of_batches):
            print("Batch: {}".format(index+1))

            # Train the Discriminator Network
            
            # Sample a batch of data for use 
            z_noise= np.random.normal(0,1,size=(batch_size,z_dim))
            image_batch= xtrain[index*batch_size:(index+1)*batch_size]
            embedding_batch= embeddings_train[index*batch_size:(index+1)*batch_size]
            image_batch= (image_batch-127.5)/127.5

            # Generate Fake Images
            fake_images,_= stage1_gen.predict([embedding_batch,z_noise], verbose=3)

            # Generate Compressed Embeddings
            compressed_embedding= embedding_compressor_model.predict_on_batch(embedding_batch)
            compressed_embedding= np.reshape(compressed_embedding, (-1,1,1,condition_dim))
            compressed_embedding= np.tile(compressed_embedding, (1,4,4,1))

            # Calculating the losses for the Discriminator network
            dis_loss_real= stage1_dis.train_on_batch([image_batch, compressed_embedding], np.reshape(real_labels, (batch_size,1)))

            dis_loss_fake= stage1_dis.train_on_batch([fake_images, compressed_embedding], np.reshape(fake_labels, (batch_size,1)))

            dis_loss_wrong= stage1_dis.train_on_batch([image_batch[:(batch_size-1)], compressed_embedding[1:]], np.reshape(fake_labels[1:], (batch_size-1,1)))

            d_loss= 0.5 * np.add(dis_loss_real, 0.5*np.add(dis_loss_wrong, dis_loss_fake))

            print("d_loss_real: {}".format(dis_loss_real))
            print("d_loss_fake: {}".format(dis_loss_fake))
            print("d_loss_wrong: {}".format(dis_loss_wrong))
            print("d_loss: {}".format(d_loss))


            # Training the Generator Network and calculating losses for it 
            g_loss= adversarial_model.train_on_batch([embedding_batch,z_noise, compressed_embedding], [K.ones((batch_size,1))*0.9, K.ones((batch_size,256))*0.9])
            print("g_loss:{}".format(g_loss))

            dis_losses.append(d_loss)
            gen_losses.append(g_loss)


        # Save losses to TensorBoard for each epoch
        write_log(tensorboard, 'discriminator_loss', np.mean(dis_losses), epoch)
        write_log(tensorboard, 'generator_loss', np.mean(gen_losses[0]), epoch)

        #Generate and Save Images after every second epoch
        if epoch%2==0:
            z_noise2= np.random.normal(0,1,size=(batch_size, z_dim))
            embedding_batch= embeddings_test[0:batch_size]
            fake_images,_= stage1_gen.predict_on_batch([embedding_batch,z_noise2])

            # Save Images
            for i,img in enumerate(fake_images[:10]):
                save_rgb_img(img, "results/gen_{}_{}.png".format(epoch,i))

    stage1_gen.save_weights("stage1_gen.h5")
    stage1_dis.save_weights("stage1_dis.h5")

Embeddings:  (8855, 10, 1024)
Embeddings Shape:  (8855, 10, 1024)
Embeddings:  (2933, 10, 1024)
Embeddings Shape:  (2933, 10, 1024)



Epoch:  0
Number of Batches:  138
Batch: 1



d_loss_real: 0.7964822053909302
d_loss_fake: 13.830137252807617
d_loss_wrong: 10.886130332946777
d_loss: 6.577307999134064



g_loss:[0.9148654937744141, 0.8812770247459412, 0.016794228926301003]
Batch: 2
d_loss_real: 2.926060199737549
d_loss_fake: 1.8676177263259888
d_loss_wrong: 0.7201905250549316
d_loss: 2.1099821627140045
g_loss:[0.9037395715713501, 0.8784630298614502, 0.012638267129659653]
Batch: 3
d_loss_real: 2.478734016418457
d_loss_fake: 6.682442665100098
d_loss_wrong: 0.5190232992172241
d_loss: 3.039733499288559
g_loss:[4.4222917556762695, 4.393670082092285, 0.014310941100120544]
Batch: 4
d_loss_real: 3.687033176422119
d_loss_fake: 2.2014596462249756
d_loss_wrong: 0.23424911499023438
d_loss: 2.452443778514862
g_loss:[2.8472018241882324, 2.8162102699279785, 0.015495788305997849]
Batch: 5
d_loss_real: 3.2246670722961426
d_loss_fake: 0.025378555059432983
d_loss_wrong: 1.7308260202407837
d_loss: 2.0513846799731255
g_loss:[0.8018344640731812, 0.7734496593475342, 0.01419239491224289]
Batch: 6
d_loss_real: 1.469877004623413
d_loss_fake: 0.2541709244251251
d_loss_wrong: 1.1531729698181152
d_loss: 1.08677447

Epoch:  1
Number of Batches:  138
Batch: 1
d_loss_real: 0.9520565867424011
d_loss_fake: 8.023594273254275e-05
d_loss_wrong: 1.0753169059753418
d_loss: 0.7448775788507191
g_loss:[0.6977493166923523, 0.684476375579834, 0.006636462174355984]
Batch: 2
d_loss_real: 0.9666059017181396
d_loss_fake: 0.0005042466800659895
d_loss_wrong: 0.8558505177497864
d_loss: 0.6973916419665329
g_loss:[0.7541218400001526, 0.7425345182418823, 0.005793665535748005]
Batch: 3
d_loss_real: 1.0512299537658691
d_loss_fake: 8.793905726633966e-05
d_loss_wrong: 0.7578851580619812
d_loss: 0.7151082511627465
g_loss:[0.8334525227546692, 0.8182675838470459, 0.007592473179101944]
Batch: 4
d_loss_real: 1.0796451568603516
d_loss_fake: 4.796412031282671e-05
d_loss_wrong: 0.7161266803741455
d_loss: 0.7188662395537904
g_loss:[0.8564842343330383, 0.8411097526550293, 0.007687244564294815]
Batch: 5
d_loss_real: 0.9500631093978882
d_loss_fake: 5.321182106854394e-05
d_loss_wrong: 0.8308354616165161
d_loss: 0.6827537230583403
g_loss:

...

Epoch:  95
Number of Batches:  138
Batch: 1
d_loss_real: 0.3684123456478119
d_loss_fake: 1.9896006051567383e-05
d_loss_wrong: 0.019288646057248116
d_loss: 0.18903330833973087
g_loss:[0.450275719165802, 0.4380888342857361, 0.006093448493629694]
Batch: 2
d_loss_real: 0.3655489981174469
d_loss_fake: 3.2326179280062206e-06
d_loss_wrong: 0.017472906038165092
d_loss: 0.18714353372274672
g_loss:[0.4468401372432709, 0.43711861968040466, 0.00486075459048152]
Batch: 3
d_loss_real: 0.4030733108520508
d_loss_fake: 9.519486638964736e-07
d_loss_wrong: 0.04529993236064911
d_loss: 0.21286187650335364
g_loss:[0.4025983214378357, 0.3912760019302368, 0.005661162547767162]
Batch: 4
d_loss_real: 0.3908069133758545
d_loss_fake: 6.014746531946003e-07
d_loss_wrong: 0.01507218461483717
d_loss: 0.19917165321029984
g_loss:[0.37783288955688477, 0.36486467719078064, 0.006484107580035925]
Batch: 5
d_loss_real: 0.3726153075695038
d_loss_fake: 1.7104891014696477e-07
d_loss_wrong: 0.015579543076455593
d_loss: 0.190202

Epoch:  97
Number of Batches:  138
Batch: 1
d_loss_real: 0.3993624746799469
d_loss_fake: 1.9763025193242356e-05
d_loss_wrong: 0.024156497791409492
d_loss: 0.20572530254412413
g_loss:[0.3695329427719116, 0.3548487424850464, 0.007342095486819744]
Batch: 2
d_loss_real: 0.3702194392681122
d_loss_fake: 2.1036948965047486e-05
d_loss_wrong: 0.015105253085494041
d_loss: 0.18889129214267086
g_loss:[0.3782699406147003, 0.3679352402687073, 0.0051673538982868195]
Batch: 3
d_loss_real: 0.4368567168712616
d_loss_fake: 2.1921539882896468e-05
d_loss_wrong: 0.050031621009111404
d_loss: 0.23094174407287937
g_loss:[0.4158965051174164, 0.40508008003234863, 0.005408219527453184]
Batch: 4
d_loss_real: 0.39461565017700195
d_loss_fake: 1.0705227850849042e-06
d_loss_wrong: 0.0334317572414875
d_loss: 0.20566603202956912
g_loss:[0.3705761432647705, 0.35589799284935, 0.00733907101675868]
Batch: 5
d_loss_real: 0.3772099018096924
d_loss_fake: 2.2166810254020675e-07
d_loss_wrong: 0.01319724228233099
d_loss: 0.191904

Epoch:  99
Number of Batches:  138
Batch: 1
d_loss_real: 0.3578188717365265
d_loss_fake: 8.87087662704289e-05
d_loss_wrong: 0.015636300668120384
d_loss: 0.18284068822686095
g_loss:[0.3907700181007385, 0.3775777816772461, 0.006596113555133343]
Batch: 2
d_loss_real: 0.3559766709804535
d_loss_fake: 3.41020459018182e-05
d_loss_wrong: 0.014683594927191734
d_loss: 0.18166775973350013
g_loss:[0.38282230496406555, 0.3725142478942871, 0.005154033191502094]
Batch: 3
d_loss_real: 0.3857588469982147
d_loss_fake: 0.00012263635289855301
d_loss_wrong: 0.05004644766449928
d_loss: 0.20542169450345682
g_loss:[0.37789320945739746, 0.366275429725647, 0.0058088842779397964]
Batch: 4
d_loss_real: 0.37784481048583984
d_loss_fake: 2.1968466171529144e-05
d_loss_wrong: 0.01592729240655899
d_loss: 0.19290972046110255
g_loss:[0.40687617659568787, 0.3913506269454956, 0.0077627780847251415]
Batch: 5
d_loss_real: 0.3724985122680664
d_loss_fake: 1.31310218876024e-06
d_loss_wrong: 0.014566725119948387
d_loss: 0.189891

Epoch:  101
Number of Batches:  138
Batch: 1
d_loss_real: 0.37040525674819946
d_loss_fake: 9.508125913271215e-06
d_loss_wrong: 0.021070487797260284
d_loss: 0.19047262735489312
g_loss:[0.3952391445636749, 0.38106080889701843, 0.007089170627295971]
Batch: 2
d_loss_real: 0.3858602046966553
d_loss_fake: 1.0173929695156403e-05
d_loss_wrong: 0.017539402469992638
d_loss: 0.19731749644824959
g_loss:[0.364533931016922, 0.3500714898109436, 0.007231224328279495]
Batch: 3
d_loss_real: 0.39448297023773193
d_loss_fake: 6.494067747553345e-06
d_loss_wrong: 0.041476067155599594
d_loss: 0.20761212542470275
g_loss:[0.3924340605735779, 0.37766435742378235, 0.007384848315268755]
Batch: 4
d_loss_real: 0.36644771695137024
d_loss_fake: 1.4648990145360585e-06
d_loss_wrong: 0.012120166793465614
d_loss: 0.18625426639880516
g_loss:[0.3817671537399292, 0.36040550470352173, 0.010680821724236012]
Batch: 5
d_loss_real: 0.3781794607639313
d_loss_fake: 5.789891588392493e-07
d_loss_wrong: 0.01273099984973669
d_loss: 0.1

KeyboardInterrupt: 